# Load libraries/packages

(... or whatever you python people call them ...)

In [1]:
%matplotlib inline
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os
import warnings
from time import gmtime, strftime
from sinfo import sinfo

In [2]:
warnings.filterwarnings('ignore')

# Start Time

In [3]:
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2022-07-15 11:00:39'

# Params

Run `param_files/scrublet_params.py` to load the settings.

In [4]:
%run -i 'param_files/scrublet_params.py'

In [ ]:
proj_dir = os.path.join(os.getcwd(),'../')
proj_dir

In [ ]:
alevin_dir = f'{proj_dir}analyses/doublets/filtered_inputs/'
alevin_dir

In [31]:
# proj_dir
# import socket
# curr_machine = socket.gethostname()
# if curr_machine == "mentok":
#     proj_dir = "/mnt/data/backups/cluster_backups/cbrg"+proj_dir
#     alevin_dir = f'{proj_dir}analyses/doublets/filtered_inputs/'
# proj_dir

### Make output directory

I imagine you'll handle this with rufus.

In [33]:
output_dir = os.path.join(proj_dir,'analyses/doublets/scrublet/')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Unzip input files

Scrublet doesn't like compressed files for input, so they need to be uncompressed. I imagine you'll handle this within rufus

In [45]:
for dir_path in glob.glob(f'{alevin_dir}*'):
    os.chdir(os.path.join(alevin_dir,dir_path))
    if os.path.isfile('matrix.mtx.gz'):
        print ("Files are still compressed")
        print ("Decompressing ... ")
        os.system('gunzip *.gz')
    else:
        print ("Files are already uncompressed.")
    os.chdir(alevin_dir)

Files are still compressed
Decompressing ... 
Files are still compressed
Decompressing ... 


### List input directories

In [46]:
os.chdir(output_dir)
myList = glob.glob(f'{alevin_dir}*')
myList

['/mnt/data/backups/cluster_backups/cbrg/project/cncb/shared/proj075/analyses/doublets/filtered_inputs/v2',
 '/mnt/data/backups/cluster_backups/cbrg/project/cncb/shared/proj075/analyses/doublets/filtered_inputs/v3']

# Run Scrublet

In [ ]:
for dir_path in myList:

    #dir_name = dir_path[78:]
    dir_name = dir_path.split('/')[-1]
    print('\n\n\n\nProcessing: ',dir_path,'\n\n')
    print('Loading data...')
    counts_matrix = scipy.io.mmread(dir_path + '/matrix.mtx').T.tocsc()
    genes = np.array(scr.load_genes(dir_path + '/features.tsv', delimiter = '\t', column = 1))
    print('Done.\n')
    print('\n\nCounts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
    print('Number of genes in gene list: {}'.format(len(genes)))
    exp_doub_rate = 0.008*counts_matrix.shape[0]/1000
    print('Expected doublet rate: ', exp_doub_rate)
    print('\n\n')

    print('Detecting doublets...')

    
    doublet_scores = np.zeros((counts_matrix.shape[0],len(pcs)))
    predicted_doublets = np.zeros((counts_matrix.shape[0],len(pcs)))
    z_scores = np.zeros((counts_matrix.shape[0],len(pcs)))
    
    for i, pc in enumerate(pcs):
        print('\n\n\n\nCurrent number of PCAs: ',pc,'\n\n')
        scrub = scr.Scrublet(counts_matrix, 
                             expected_doublet_rate = exp_doub_rate, 
                             sim_doublet_ratio = sim_doub_rat)
        temp_scores, temp_doublets = scrub.scrub_doublets(min_counts = min_count, 
                                                          min_cells = 3, 
                                                          min_gene_variability_pctl = vgp, 
                                                          n_prin_comps = pc,
                                                          log_transform = log_trans,
                                                          mean_center = mean_cent,
                                                          normalize_variance = norm_var,
                                                          synthetic_doublet_umi_subsampling = syn_doub_umi)
        
        temp_doublets = scrub.call_doublets(threshold = my_thresh)
        temp_z_scores = scrub.z_scores_
        
        doublet_scores[:,i] = temp_scores
        predicted_doublets[:,i] = temp_doublets
        z_scores[:,i] = temp_z_scores
        
        print(f'\nIdentified {temp_doublets[temp_doublets == True].shape[0]} doublets...\n\n')
        %matplotlib inline
        p1 = scrub.plot_histogram();
        plt.show()
        print('Running UMAP...')
        scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist = 0.3))
        print('Plotting UMAP...')
        print('    Predicted Doublets and Doublet Score')
        %matplotlib inline
        p2 = scrub.plot_embedding('UMAP', order_points = True);
        plt.show()
        print('    Predicted Doublets and Z-Score')
        %matplotlib inline
        p3 = scrub.plot_embedding('UMAP', order_points = True, score = 'zscore');
        plt.show()
        print('Done.\n')

    print('Done.\n')

    barcodes = pd.read_csv(f'{dir_path}/barcodes.tsv',sep = "\t",header = None)    
    
    doublet_scores_df = pd.DataFrame(doublet_scores, index = barcodes[0], columns = pcs)
    z_scores_df = pd.DataFrame(z_scores, index = barcodes[0], columns = pcs)
    predicted_doublets_df = pd.DataFrame(predicted_doublets, index = barcodes[0], columns = pcs)
    
    
    print('Saving data...')
    doublet_scores_df.to_csv(f'{dir_name}_doublet_scores.csv', index = True, header = True, sep = ',')
    z_scores_df.to_csv(f'{dir_name}_z_scores.csv', index = True, header = True, sep = ',')
    predicted_doublets_df.to_csv(f'{dir_name}_predicted_doublets.csv', index = True, header = True, sep = ',') 
    print('Done.\n\n\n\n')


    

# Session Info

In [ ]:
sinfo()

# Stop Time

In [ ]:
strftime("%Y-%m-%d %H:%M:%S", gmtime())